# [Pyomo.GDP](./index.ipynb) Logical Expression System Demo - Working with Disjuncts and Disjunctions

The logical expression system is designed to augment the previously introduced `Disjunct` and `Disjunction` Pyomo components, the first components in Pyomo.GDP.
These components allow the organization of numeric constraints into logical contexts, which we term disjuncts.

All of the constraints in a disjuncts are all either enforced, or left unenforced.
Note that constraints left unenforced may still be satisfied.
An indicator variable denotes whether the constraints of a disjunct are enforced.
The groupings of constraints organized in disjuncts may then be related to each other by OR relationships, i.e. disjunctions.

In literature, the disjunct indicator variable is Boolean; however, for historical reasons, it was originally implemented in Pyomo.GDP as a binary variable.
The logical expression system allows us to now associate a Boolean variable to the disjuncts and declare proper logical propositions involving disjuncts.
These logical propositions are associated with our Pyomo model using `LogicalStatement` objects.

The code currently relies on the logic-v1 branch at https://github.com/qtothec/pyomo/tree/logic-v1.

In [1]:
from pyomo.environ import *
from pyomo.gdp import *
from pyomo.core.expr.logical_expr import *
from pyomo.core.plugins.transform.logical_to_linear import update_boolean_vars_from_binary

Here, we define a toy example to demonstrate the capability:
    
$$\begin{aligned}\min~&x\\
\text{s.t.}~&\left[\begin{gathered}Y_1\\x \geq 2\end{gathered}\right] \vee \left[\begin{gathered}Y_2\\x \geq 3\end{gathered}\right]\\
&\left[\begin{gathered}Y_3\\x \leq 8\end{gathered}\right] \vee \left[\begin{gathered}Y_4\\x = 2.5\end{gathered}\right] \\
&Y_1 \underline{\vee} Y_2\\
&Y_3 \underline{\vee} Y_4\\
&Y_1 \Rightarrow Y_4
\end{aligned}$$

In [2]:
def build_new_model():
    m = ConcreteModel()
    m.s = RangeSet(4)
    m.ds = RangeSet(2)
    m.Y = BooleanVar(m.s)
    m.d = Disjunct(m.s)
    m.djn = Disjunction(m.ds)
    m.djn[1] = [m.d[1], m.d[2]]
    m.djn[2] = [m.d[3], m.d[4]]
    m.x = Var(bounds=(-2, 10))
    m.d[1].c = Constraint(expr=m.x >= 2)
    m.d[2].c = Constraint(expr=m.x >= 3)
    m.d[3].c = Constraint(expr=m.x <= 8)
    m.d[4].c = Constraint(expr=m.x == 2.5)
    m.o = Objective(expr=m.x)
    return m

In [3]:
m = build_new_model()
# Associate Boolean vars with auto-generated (by Pyomo.GDP) binaries
for i in m.s:
    m.Y[i].set_binary_var(m.d[i].indicator_var)

Add the implication $Y_1 \Rightarrow Y_4$

In [4]:
m.p = LogicalStatement(expr=m.Y[1].implies(m.Y[4]))
# Note: the implicit XOR enforced by the Disjunction object defined above still applies

In [5]:
TransformationFactory('core.logical_to_linear').apply_to(m)
TransformationFactory('gdp.bigm').apply_to(m)

In [6]:
m.Y.display()  # Before solve, Boolean vars have no value

Y : Size=4, Index=s
    Key : Value : Fixed : Stale
      1 :  None : False :  True
      2 :  None : False :  True
      3 :  None : False :  True
      4 :  None : False :  True


In [7]:
SolverFactory('gams').solve(m)
update_boolean_vars_from_binary(m)

In [8]:
# m.display()
m.Y.display()

Y : Size=4, Index=s
    Key : Value : Fixed : Stale
      1 :  True : False :  True
      2 : False : False :  True
      3 : False : False :  True
      4 :  True : False :  True


## Logical statements on Disjuncts

The new expression system lets modelers attach logical statements onto Disjuncts. This is not part of the classic literature GDP formulation, but can be understood as an implication relationship. That is, we have the following disjunctive structure, which resolves to the same logic as above:

$$\begin{aligned}\min~&x\\
\text{s.t.}~&\left[\begin{gathered}Y_1\\x \geq 2\\Y_4 = True\end{gathered}\right] \vee \left[\begin{gathered}Y_2\\x \geq 3\end{gathered}\right]\\
&\left[\begin{gathered}Y_3\\x \leq 8\end{gathered}\right] \vee \left[\begin{gathered}Y_4\\x = 2.5\end{gathered}\right] \\
&Y_1 \underline{\vee} Y_2\\
&Y_3 \underline{\vee} Y_4
\end{aligned}$$

Demonstrating this on our earlier example, we have:

In [11]:
m = build_new_model()
for i in m.s:
    m.Y[i].set_binary_var(m.d[i].indicator_var)
    
m.d[1].p = LogicalStatement(expr=m.Y[4])
TransformationFactory('core.logical_to_linear').apply_to(m)
TransformationFactory('gdp.bigm').apply_to(m)
SolverFactory('gams').solve(m)
update_boolean_vars_from_binary(m)
m.Y.display()

Y : Size=4, Index=s
    Key : Value : Fixed : Stale
      1 :  True : False :  True
      2 : False : False :  True
      3 : False : False :  True
      4 :  True : False :  True


## Indexed logical statements

Like `Constraint` objects for numerical expressions, `LogicalStatement` objects can be used in an indexed manner.
An example of this usage may be found below for the expression:

$$Y_{i+1} \Rightarrow Y_{i}, \quad \forall i \in \{1, 2, \dots, n-1\}$$

In [12]:
m = ConcreteModel()
n = 5
m.I = RangeSet(n)
m.Y = BooleanVar(m.I)

@m.LogicalStatement(m.I)
def p(m, i):
    return m.Y[i+1] >> m.Y[i] if i < n else True

m.pprint()

1 RangeSet Declarations
    I : Dim=0, Dimen=1, Size=5, Domain=Integers, Ordered=True, Bounds=(1, 5)
        Virtual

1 BooleanVar Declarations
    Y : Size=5, Index=I
        Key : Value : Fixed : Stale
          1 :  None : False :  True
          2 :  None : False :  True
          3 :  None : False :  True
          4 :  None : False :  True
          5 :  None : False :  True

1 LogicalStatement Declarations
    p : Size=4, Index=I, Active=True
        Key : Body         : Active
          1 : Y[2] >> Y[1] :   True
          2 : Y[3] >> Y[2] :   True
          3 : Y[4] >> Y[3] :   True
          4 : Y[5] >> Y[4] :   True

3 Declarations: I Y p


In [13]:
TransformationFactory('core.logical_to_linear').apply_to(m)
m.p.pprint()
m.logic_to_linear.pprint()

p : Size=4, Index=I, Active=True
    Key : Body         : Active
      1 : Y[2] >> Y[1] :  False
      2 : Y[3] >> Y[2] :  False
      3 : Y[4] >> Y[3] :  False
      4 : Y[5] >> Y[4] :  False
logic_to_linear : Size=4, Index=logic_to_linear_index, Active=True
    Key : Lower : Body                              : Upper : Active
      1 :   1.0 : 1 - Y_asbinary[2] + Y_asbinary[1] :  +Inf :   True
      2 :   1.0 : 1 - Y_asbinary[3] + Y_asbinary[2] :  +Inf :   True
      3 :   1.0 : 1 - Y_asbinary[4] + Y_asbinary[3] :  +Inf :   True
      4 :   1.0 : 1 - Y_asbinary[5] + Y_asbinary[4] :  +Inf :   True
